<a href="https://colab.research.google.com/github/pvvkishore/NLPA_SKILL_2025/blob/main/Text_Classification_Pipeline_ANNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter, defaultdict
import pickle
import json
import re
from typing import List, Dict, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

class TextDataset:
    """
    Create and manage a text classification dataset
    """

    def __init__(self):
        self.create_dataset()
        self.display_dataset_characteristics()

    def create_dataset(self):
        """Create a 3-class sentiment classification dataset"""

        print("="*80)
        print("STEP 1: CREATING TEXT CLASSIFICATION DATASET")
        print("="*80)

        # Create training data
        self.train_data = [
            # Positive sentiment
            ("I love this movie it's absolutely amazing", "positive"),
            ("This is the best product I have ever bought", "positive"),
            ("Fantastic experience wonderful service", "positive"),
            ("Great quality excellent value for money", "positive"),
            ("Perfect exactly what I needed highly recommend", "positive"),
            ("Outstanding performance superb design", "positive"),
            ("Brilliant idea love the innovation", "positive"),
            ("Excellent customer service very helpful", "positive"),
            ("Amazing results exceeded my expectations", "positive"),
            ("Beautiful design high quality materials", "positive"),
            ("Wonderful experience great staff", "positive"),
            ("Perfect solution exactly what I wanted", "positive"),

            # Negative sentiment
            ("This movie is terrible waste of time", "negative"),
            ("Worst product ever complete disappointment", "negative"),
            ("Horrible service never buying again", "negative"),
            ("Poor quality broke after one day", "negative"),
            ("Terrible experience bad customer service", "negative"),
            ("Awful design very uncomfortable", "negative"),
            ("Disappointing results not worth the money", "negative"),
            ("Bad quality cheap materials", "negative"),
            ("Horrible experience rude staff", "negative"),
            ("Terrible product poor performance", "negative"),
            ("Worst service ever experienced", "negative"),
            ("Complete waste of money very disappointed", "negative"),

            # Neutral sentiment
            ("The product is okay nothing special", "neutral"),
            ("Average service acceptable quality", "neutral"),
            ("It works fine basic functionality", "neutral"),
            ("Standard product meets basic requirements", "neutral"),
            ("Decent quality reasonable price", "neutral"),
            ("Normal experience as expected", "neutral"),
            ("Regular service standard quality", "neutral"),
            ("It's fine nothing extraordinary", "neutral"),
            ("Average performance acceptable results", "neutral"),
            ("Standard design basic features", "neutral"),
            ("Normal quality adequate for the price", "neutral"),
            ("Acceptable product does the job", "neutral"),
        ]

        # Create test data
        self.test_data = [
            ("This is amazing I love it", "positive"),
            ("Terrible quality very disappointed", "negative"),
            ("It's okay nothing special", "neutral"),
            ("Excellent product highly recommend", "positive"),
            ("Worst experience ever", "negative"),
            ("Standard quality acceptable", "neutral"),
        ]

        # Create inference data (unlabeled)
        self.inference_data = [
            "This product is fantastic",
            "Really bad experience",
            "It's just average",
            "Outstanding quality",
            "Poor service",
            "Normal product"
        ]

        print(f"✅ Dataset created successfully!")
        print(f"   Training samples: {len(self.train_data)}")
        print(f"   Test samples: {len(self.test_data)}")
        print(f"   Inference samples: {len(self.inference_data)}")

    def display_dataset_characteristics(self):
        """Display comprehensive dataset analysis"""

        print(f"\n📊 DATASET CHARACTERISTICS ANALYSIS")
        print("-"*50)

        # Extract texts and labels
        train_texts, train_labels = zip(*self.train_data)
        test_texts, test_labels = zip(*self.test_data)

        # Class distribution
        train_label_counts = Counter(train_labels)
        test_label_counts = Counter(test_labels)

        print(f"🏷️  CLASS DISTRIBUTION:")
        print(f"   Training set:")
        for label, count in train_label_counts.items():
            percentage = (count / len(train_labels)) * 100
            print(f"     {label}: {count} samples ({percentage:.1f}%)")

        print(f"   Test set:")
        for label, count in test_label_counts.items():
            percentage = (count / len(test_labels)) * 100
            print(f"     {label}: {count} samples ({percentage:.1f}%)")

        # Text length analysis
        train_lengths = [len(text.split()) for text in train_texts]
        test_lengths = [len(text.split()) for text in test_texts]

        print(f"\n📏 TEXT LENGTH ANALYSIS:")
        print(f"   Training set - Min: {min(train_lengths)}, Max: {max(train_lengths)}, Avg: {np.mean(train_lengths):.1f}")
        print(f"   Test set - Min: {min(test_lengths)}, Max: {max(test_lengths)}, Avg: {np.mean(test_lengths):.1f}")

        # Vocabulary analysis
        all_words = []
        for text in train_texts:
            all_words.extend(text.lower().split())

        vocab_size = len(set(all_words))
        most_common = Counter(all_words).most_common(10)

        print(f"\n📚 VOCABULARY ANALYSIS:")
        print(f"   Total words: {len(all_words)}")
        print(f"   Unique words: {vocab_size}")
        print(f"   Most common words: {most_common[:5]}")

        # Sample data display
        print(f"\n📋 SAMPLE DATA STRUCTURE:")
        print(f"   Training samples:")
        for i, (text, label) in enumerate(self.train_data[:3]):
            print(f"     {i+1}. Text: '{text}'")
            print(f"        Label: '{label}'")

        return {
            'train_samples': len(self.train_data),
            'test_samples': len(self.test_data),
            'num_classes': len(train_label_counts),
            'vocab_size': vocab_size,
            'avg_length': np.mean(train_lengths),
            'class_distribution': dict(train_label_counts)
        }

class TextPreprocessor:
    """
    Handle text preprocessing and vocabulary building
    """

    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.vocab_size = 0

    def build_vocabulary(self, texts: List[str]) -> None:
        """Build vocabulary from training texts"""

        print(f"\n" + "="*80)
        print("STEP 2: TEXT PREPROCESSING & VOCABULARY BUILDING")
        print("="*80)

        # Collect all words
        all_words = []
        for text in texts:
            # Basic preprocessing
            text = text.lower()
            text = re.sub(r'[^a-zA-Z\s]', '', text)
            words = text.split()
            all_words.extend(words)

        # Create vocabulary
        word_counts = Counter(all_words)

        # Add special tokens
        self.word2idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx2word = {0: '<PAD>', 1: '<UNK>'}

        # Add words to vocabulary
        for word, count in word_counts.items():
            if count >= 1:  # Keep all words for this small dataset
                idx = len(self.word2idx)
                self.word2idx[word] = idx
                self.idx2word[idx] = word

        self.vocab_size = len(self.word2idx)

        print(f"✅ Vocabulary built successfully!")
        print(f"   Vocabulary size: {self.vocab_size}")
        print(f"   Most common words: {word_counts.most_common(10)}")

        # Display word2idx mapping (first 15 entries)
        print(f"\n🗂️  WORD2IDX MAPPING (first 15 entries):")
        for i, (word, idx) in enumerate(list(self.word2idx.items())[:15]):
            print(f"   '{word}' → {idx}")

        # Display idx2word mapping (first 15 entries)
        print(f"\n🗂️  IDX2WORD MAPPING (first 15 entries):")
        for i, (idx, word) in enumerate(list(self.idx2word.items())[:15]):
            print(f"   {idx} → '{word}'")

    def text_to_sequence(self, text: str, max_length: int = 10) -> List[int]:
        """Convert text to sequence of indices"""
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        words = text.split()

        # Convert words to indices
        sequence = []
        for word in words:
            if word in self.word2idx:
                sequence.append(self.word2idx[word])
            else:
                sequence.append(self.word2idx['<UNK>'])

        # Pad or truncate to max_length
        if len(sequence) < max_length:
            sequence.extend([self.word2idx['<PAD>']] * (max_length - len(sequence)))
        else:
            sequence = sequence[:max_length]

        return sequence

    def sequence_to_text(self, sequence: List[int]) -> str:
        """Convert sequence of indices back to text"""
        words = []
        for idx in sequence:
            if idx in self.idx2word and self.idx2word[idx] != '<PAD>':
                words.append(self.idx2word[idx])
        return ' '.join(words)

    def process_dataset(self, data: List[Tuple[str, str]], max_length: int = 10):
        """Process entire dataset"""

        print(f"\n📝 PROCESSING DATASET:")
        print(f"   Max sequence length: {max_length}")

        # Create label mapping
        labels = [label for _, label in data]
        unique_labels = sorted(list(set(labels)))
        label2idx = {label: idx for idx, label in enumerate(unique_labels)}
        idx2label = {idx: label for label, idx in label2idx.items()}

        print(f"   Label mapping: {label2idx}")

        # Process texts and labels
        sequences = []
        label_indices = []

        print(f"\n🔄 CONVERSION EXAMPLES:")
        for i, (text, label) in enumerate(data[:3]):
            sequence = self.text_to_sequence(text, max_length)
            label_idx = label2idx[label]

            sequences.append(sequence)
            label_indices.append(label_idx)

            print(f"   Example {i+1}:")
            print(f"     Original: '{text}' → {label}")
            print(f"     Sequence: {sequence}")
            print(f"     Label idx: {label_idx}")
            print(f"     Back to text: '{self.sequence_to_text(sequence)}'")

        # Process all data
        for text, label in data[3:]:
            sequence = self.text_to_sequence(text, max_length)
            label_idx = label2idx[label]
            sequences.append(sequence)
            label_indices.append(label_idx)

        return np.array(sequences), np.array(label_indices), label2idx, idx2label

class TextClassifierDataset(Dataset):
    """PyTorch Dataset for text classification"""

    def __init__(self, sequences, labels):
        self.sequences = torch.LongTensor(sequences)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

class TextClassifierNN(nn.Module):
    """
    Simple Neural Network for Text Classification
    """

    def __init__(self, vocab_size: int, embedding_dim: int, hidden_dim: int, num_classes: int):
        super(TextClassifierNN, self).__init__()

        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Neural network layers
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(hidden_dim // 2, num_classes)

        print(f"\n🏗️  NEURAL NETWORK ARCHITECTURE:")
        print(f"   Embedding Layer: {vocab_size} → {embedding_dim}")
        print(f"   Hidden Layer 1: {embedding_dim} → {hidden_dim}")
        print(f"   Hidden Layer 2: {hidden_dim} → {hidden_dim // 2}")
        print(f"   Output Layer: {hidden_dim // 2} → {num_classes}")
        print(f"   Total parameters: {sum(p.numel() for p in self.parameters()):,}")

    def forward(self, x):
        # x shape: (batch_size, sequence_length)

        # Embedding
        embedded = self.embedding(x)  # (batch_size, seq_length, embedding_dim)

        # Average pooling over sequence length
        pooled = torch.mean(embedded, dim=1)  # (batch_size, embedding_dim)

        # Neural network
        x = F.relu(self.fc1(pooled))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)

        return x

    def get_embeddings(self, sequences):
        """Get embedding vectors for sequences"""
        with torch.no_grad():
            embedded = self.embedding(sequences)
            pooled = torch.mean(embedded, dim=1)
            return pooled.numpy()

class ModelTrainer:
    """
    Handle model training and evaluation
    """

    def __init__(self, model, train_loader, test_loader, num_classes):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.num_classes = num_classes

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(model.parameters(), lr=0.001)

        self.train_losses = []
        self.train_accuracies = []
        self.test_accuracies = []

        # Store weights at different stages
        self.initial_weights = {}
        self.middle_weights = {}
        self.final_weights = {}

    def save_weights(self, stage: str):
        """Save model weights at different training stages"""
        weights = {}
        for name, param in self.model.named_parameters():
            weights[name] = param.data.clone()

        if stage == 'initial':
            self.initial_weights = weights
        elif stage == 'middle':
            self.middle_weights = weights
        elif stage == 'final':
            self.final_weights = weights

    def train_epoch(self, epoch):
        """Train for one epoch"""
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0

        for batch_idx, (data, target) in enumerate(self.train_loader):
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = self.criterion(output, target)
            loss.backward()
            self.optimizer.step()

            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)

        avg_loss = total_loss / len(self.train_loader)
        accuracy = 100. * correct / total

        self.train_losses.append(avg_loss)
        self.train_accuracies.append(accuracy)

        return avg_loss, accuracy

    def evaluate(self):
        """Evaluate model on test set"""
        self.model.eval()
        correct = 0
        total = 0
        all_preds = []
        all_targets = []

        with torch.no_grad():
            for data, target in self.test_loader:
                output = self.model(data)
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total += target.size(0)

                all_preds.extend(pred.cpu().numpy())
                all_targets.extend(target.cpu().numpy())

        accuracy = 100. * correct / total
        self.test_accuracies.append(accuracy)

        return accuracy, all_preds, all_targets

    def train(self, num_epochs: int):
        """Complete training process"""

        print(f"\n" + "="*80)
        print("STEP 4: NEURAL NETWORK TRAINING")
        print("="*80)

        # Save initial weights
        self.save_weights('initial')

        print(f"🚀 Starting training for {num_epochs} epochs...")
        print(f"   Optimizer: Adam (lr=0.001)")
        print(f"   Loss function: CrossEntropyLoss")
        print(f"   Batch size: {self.train_loader.batch_size}")

        print(f"\n📊 TRAINING PROGRESS:")
        print(f"{'Epoch':<6} {'Train Loss':<12} {'Train Acc':<12} {'Test Acc':<12}")
        print("-" * 50)

        for epoch in range(num_epochs):
            train_loss, train_acc = self.train_epoch(epoch)
            test_acc, _, _ = self.evaluate()

            # Save middle weights at halfway point
            if epoch == num_epochs // 2:
                self.save_weights('middle')

            print(f"{epoch+1:<6} {train_loss:<12.4f} {train_acc:<12.1f}% {test_acc:<12.1f}%")

        # Save final weights
        self.save_weights('final')

        print(f"\n✅ Training completed!")
        print(f"   Final train accuracy: {self.train_accuracies[-1]:.1f}%")
        print(f"   Final test accuracy: {self.test_accuracies[-1]:.1f}%")

    def display_weight_analysis(self):
        """Display weight changes during training"""

        print(f"\n🔍 WEIGHT ANALYSIS:")
        print("-" * 50)

        # Analyze embedding weights
        emb_initial = self.initial_weights['embedding.weight']
        emb_middle = self.middle_weights['embedding.weight']
        emb_final = self.final_weights['embedding.weight']

        print(f"📊 Embedding Layer Weight Changes:")
        print(f"   Initial weights (first 5 words, first 3 dims):")
        print(f"   {emb_initial[:5, :3].numpy()}")
        print(f"   Final weights (first 5 words, first 3 dims):")
        print(f"   {emb_final[:5, :3].numpy()}")

        # Calculate weight change magnitude
        weight_change = torch.norm(emb_final - emb_initial).item()
        print(f"   Total embedding weight change magnitude: {weight_change:.4f}")

        # Analyze first linear layer
        fc1_initial = self.initial_weights['fc1.weight']
        fc1_final = self.final_weights['fc1.weight']
        fc1_change = torch.norm(fc1_final - fc1_initial).item()

        print(f"\n📊 First Linear Layer Weight Changes:")
        print(f"   Weight change magnitude: {fc1_change:.4f}")
        print(f"   Initial weights (first 3x3):")
        print(f"   {fc1_initial[:3, :3].numpy()}")
        print(f"   Final weights (first 3x3):")
        print(f"   {fc1_final[:3, :3].numpy()}")

def display_embeddings(model, preprocessor, sample_texts, idx2label):
    """Display embedding vectors for sample texts"""

    print(f"\n" + "="*80)
    print("STEP 5: EMBEDDING ANALYSIS")
    print("="*80)

    print(f"🔤 EMBEDDING VECTORS FOR SAMPLE TEXTS:")

    for i, text in enumerate(sample_texts[:3]):
        sequence = preprocessor.text_to_sequence(text)
        sequence_tensor = torch.LongTensor([sequence])

        # Get embedding
        embeddings = model.get_embeddings(sequence_tensor)
        embedding_vector = embeddings[0]

        print(f"\n   Text {i+1}: '{text}'")
        print(f"   Sequence: {sequence}")
        print(f"   Embedding vector (first 10 dims): {embedding_vector[:10]}")
        print(f"   Embedding shape: {embedding_vector.shape}")
        print(f"   Embedding magnitude: {np.linalg.norm(embedding_vector):.4f}")

def plot_training_metrics(trainer):
    """Plot training metrics"""

    print(f"\n📈 PLOTTING TRAINING METRICS...")

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

    # Training loss
    ax1.plot(trainer.train_losses, 'b-', linewidth=2)
    ax1.set_title('Training Loss Over Time')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.grid(True, alpha=0.3)

    # Training accuracy
    ax2.plot(trainer.train_accuracies, 'g-', linewidth=2, label='Train')
    ax2.plot(trainer.test_accuracies, 'r-', linewidth=2, label='Test')
    ax2.set_title('Accuracy Over Time')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # Weight change visualization
    epochs = range(len(trainer.train_losses))
    ax3.bar(epochs, trainer.train_losses, alpha=0.7, color='skyblue')
    ax3.set_title('Loss Distribution by Epoch')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Loss')

    # Performance summary
    ax4.text(0.1, 0.8, f"Final Training Accuracy: {trainer.train_accuracies[-1]:.1f}%",
             fontsize=12, transform=ax4.transAxes)
    ax4.text(0.1, 0.6, f"Final Test Accuracy: {trainer.test_accuracies[-1]:.1f}%",
             fontsize=12, transform=ax4.transAxes)
    ax4.text(0.1, 0.4, f"Best Test Accuracy: {max(trainer.test_accuracies):.1f}%",
             fontsize=12, transform=ax4.transAxes)
    ax4.text(0.1, 0.2, f"Total Epochs: {len(trainer.train_losses)}",
             fontsize=12, transform=ax4.transAxes)
    ax4.set_title('Training Summary')
    ax4.axis('off')

    plt.tight_layout()
    plt.savefig('training_metrics.png', dpi=300, bbox_inches='tight')
    plt.show()

def test_model_inference(model, preprocessor, test_data, inference_data, idx2label):
    """Test model on new data and show inference process"""

    print(f"\n" + "="*80)
    print("STEP 6: MODEL TESTING & INFERENCE")
    print("="*80)

    model.eval()

    # Test on labeled test data
    print(f"🧪 TESTING ON LABELED DATA:")
    print("-" * 40)

    all_predictions = []
    all_true_labels = []

    for i, (text, true_label) in enumerate(test_data):
        sequence = preprocessor.text_to_sequence(text)
        sequence_tensor = torch.LongTensor([sequence])

        with torch.no_grad():
            output = model(sequence_tensor)
            probabilities = F.softmax(output, dim=1)
            predicted_idx = output.argmax(dim=1).item()
            predicted_label = idx2label[predicted_idx]
            confidence = probabilities[0][predicted_idx].item()

        all_predictions.append(predicted_idx)
        all_true_labels.append(list(idx2label.values()).index(true_label))

        print(f"   Test {i+1}:")
        print(f"     Text: '{text}'")
        print(f"     True label: {true_label}")
        print(f"     Predicted: {predicted_label}")
        print(f"     Confidence: {confidence:.3f}")
        print(f"     Probabilities: {dict(zip(idx2label.values(), probabilities[0].numpy()))}")
        print()

    # Calculate test metrics
    test_accuracy = accuracy_score(all_true_labels, all_predictions)
    print(f"📊 TEST RESULTS:")
    print(f"   Test Accuracy: {test_accuracy:.3f} ({test_accuracy*100:.1f}%)")

    # Classification report
    print(f"\n📋 DETAILED CLASSIFICATION REPORT:")
    report = classification_report(all_true_labels, all_predictions,
                                 target_names=list(idx2label.values()),
                                 zero_division=0)
    print(report)

    # Confusion matrix
    cm = confusion_matrix(all_true_labels, all_predictions)
    print(f"🔢 CONFUSION MATRIX:")
    print(f"   {cm}")

    # Test on inference data (unlabeled)
    print(f"\n🔮 INFERENCE ON NEW DATA:")
    print("-" * 40)

    for i, text in enumerate(inference_data):
        sequence = preprocessor.text_to_sequence(text)
        sequence_tensor = torch.LongTensor([sequence])

        with torch.no_grad():
            output = model(sequence_tensor)
            probabilities = F.softmax(output, dim=1)
            predicted_idx = output.argmax(dim=1).item()
            predicted_label = idx2label[predicted_idx]
            confidence = probabilities[0][predicted_idx].item()

        print(f"   Inference {i+1}:")
        print(f"     Text: '{text}'")
        print(f"     Predicted: {predicted_label}")
        print(f"     Confidence: {confidence:.3f}")
        print(f"     All probabilities:")
        for label, prob in zip(idx2label.values(), probabilities[0].numpy()):
            print(f"       {label}: {prob:.3f}")
        print()

def save_model_and_artifacts(model, preprocessor, idx2label, trainer):
    """Save trained model and related artifacts"""

    print(f"\n💾 SAVING MODEL AND ARTIFACTS:")

    # Save model
    torch.save(model.state_dict(), 'text_classifier_model.pth')
    print(f"   ✅ Model saved: text_classifier_model.pth")

    # Save preprocessor
    with open('preprocessor.pkl', 'wb') as f:
        pickle.dump(preprocessor, f)
    print(f"   ✅ Preprocessor saved: preprocessor.pkl")

    # Save label mappings
    with open('label_mappings.json', 'w') as f:
        json.dump({'idx2label': idx2label}, f, indent=2)
    print(f"   ✅ Label mappings saved: label_mappings.json")

    # Save training history
    training_history = {
        'train_losses': trainer.train_losses,
        'train_accuracies': trainer.train_accuracies,
        'test_accuracies': trainer.test_accuracies
    }
    with open('training_history.json', 'w') as f:
        json.dump(training_history, f, indent=2)
    print(f"   ✅ Training history saved: training_history.json")

def main():
    """Main function to run the complete text classification pipeline"""

    print("🚀 COMPLETE TEXT CLASSIFICATION NEURAL NETWORK TUTORIAL")
    print("="*80)

    # Step 1: Create dataset
    dataset = TextDataset()

    # Step 2: Build vocabulary and preprocess
    preprocessor = TextPreprocessor()
    train_texts = [text for text, _ in dataset.train_data]
    preprocessor.build_vocabulary(train_texts)

    # Process datasets
    max_length = 10
    train_sequences, train_labels, label2idx, idx2label = preprocessor.process_dataset(
        dataset.train_data, max_length)
    test_sequences, test_labels, _, _ = preprocessor.process_dataset(
        dataset.test_data, max_length)

    print(f"\n🔄 DATASET PROCESSING COMPLETE:")
    print(f"   Training sequences shape: {train_sequences.shape}")
    print(f"   Training labels shape: {train_labels.shape}")
    print(f"   Test sequences shape: {test_sequences.shape}")
    print(f"   Test labels shape: {test_labels.shape}")

    # Step 3: Create data loaders
    print(f"\n" + "="*80)
    print("STEP 3: CREATING DATA LOADERS")
    print("="*80)

    train_dataset = TextClassifierDataset(train_sequences, train_labels)
    test_dataset = TextClassifierDataset(test_sequences, test_labels)

    batch_size = 8
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"✅ Data loaders created:")
    print(f"   Training batches: {len(train_loader)}")
    print(f"   Test batches: {len(test_loader)}")
    print(f"   Batch size: {batch_size}")

    # Display sample batch
    sample_batch = next(iter(train_loader))
    sample_sequences, sample_labels = sample_batch
    print(f"\n📦 SAMPLE BATCH:")
    print(f"   Batch sequences shape: {sample_sequences.shape}")
    print(f"   Batch labels shape: {sample_labels.shape}")
    print(f"   Sample sequence: {sample_sequences[0].numpy()}")
    print(f"   Sample label: {sample_labels[0].item()} ({idx2label[sample_labels[0].item()]})")

    # Initialize model
    vocab_size = preprocessor.vocab_size
    embedding_dim = 16
    hidden_dim = 32
    num_classes = len(label2idx)

    model = TextClassifierNN(vocab_size, embedding_dim, hidden_dim, num_classes)

    # Display model architecture details
    print(f"\n🏗️  MODEL ARCHITECTURE DETAILS:")
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"   Total parameters: {total_params:,}")
    print(f"   Trainable parameters: {trainable_params:,}")

    # Show layer details
    for name, module in model.named_modules():
        if len(list(module.children())) == 0:  # Leaf modules only
            params = sum(p.numel() for p in module.parameters())
            print(f"   {name}: {params:,} parameters")

    # Initialize trainer
    trainer = ModelTrainer(model, train_loader, test_loader, num_classes)

    # Train model
    num_epochs = 20
    trainer.train(num_epochs)

    # Display weight analysis
    trainer.display_weight_analysis()

    # Display embeddings
    sample_texts = [text for text, _ in dataset.train_data[:3]]
    display_embeddings(model, preprocessor, sample_texts, idx2label)

    # Plot training metrics
    plot_training_metrics(trainer)

    # Test model
    test_model_inference(model, preprocessor, dataset.test_data,
                        dataset.inference_data, idx2label)

    # Save everything
    save_model_and_artifacts(model, preprocessor, idx2label, trainer)

    print(f"\n" + "="*80)
    print("🎉 TUTORIAL COMPLETE!")
    print("="*80)
    print(f"✅ Successfully trained a text classification neural network!")
    print(f"✅ Model achieved {trainer.test_accuracies[-1]:.1f}% test accuracy")
    print(f"✅ All artifacts saved for future use")
    print(f"✅ Visualizations generated")

    print(f"\n📚 WHAT THE MODEL LEARNED:")
    print("-" * 40)
    print(f"🎯 The neural network learned to:")
    print(f"   • Map words to meaningful vector representations")
    print(f"   • Distinguish between positive, negative, and neutral sentiments")
    print(f"   • Generalize from training examples to new text")
    print(f"   • Assign confidence scores to predictions")

    print(f"\n🔍 KEY INSIGHTS:")
    print(f"   • Embedding vectors capture semantic meaning")
    print(f"   • Training gradually adjusts weights to improve accuracy")
    print(f"   • Model confidence indicates prediction reliability")
    print(f"   • Performance metrics help evaluate model quality")

    print(f"\n🚀 NEXT STEPS:")
    print(f"   • Try larger datasets for better performance")
    print(f"   • Experiment with different architectures (LSTM, Transformer)")
    print(f"   • Use pre-trained embeddings (Word2Vec, GloVe)")
    print(f"   • Apply to real-world text classification problems")

def load_and_use_saved_model():
    """Demonstrate how to load and use the saved model"""

    print(f"\n" + "="*80)
    print("BONUS: LOADING AND USING SAVED MODEL")
    print("="*80)

    try:
        # Load preprocessor
        with open('preprocessor.pkl', 'rb') as f:
            preprocessor = pickle.load(f)

        # Load label mappings
        with open('label_mappings.json', 'r') as f:
            mappings = json.load(f)
            idx2label = {int(k): v for k, v in mappings['idx2label'].items()}

        # Initialize model with same architecture
        vocab_size = preprocessor.vocab_size
        embedding_dim = 16
        hidden_dim = 32
        num_classes = len(idx2label)

        model = TextClassifierNN(vocab_size, embedding_dim, hidden_dim, num_classes)

        # Load trained weights
        model.load_state_dict(torch.load('text_classifier_model.pth'))
        model.eval()

        print(f"✅ Model loaded successfully!")

        # Test on new examples
        new_texts = [
            "This is absolutely fantastic!",
            "Really disappointing experience",
            "It's okay, nothing special"
        ]

        print(f"\n🔮 PREDICTIONS ON NEW TEXT:")
        for text in new_texts:
            sequence = preprocessor.text_to_sequence(text)
            sequence_tensor = torch.LongTensor([sequence])

            with torch.no_grad():
                output = model(sequence_tensor)
                probabilities = F.softmax(output, dim=1)
                predicted_idx = output.argmax(dim=1).item()
                predicted_label = idx2label[predicted_idx]
                confidence = probabilities[0][predicted_idx].item()

            print(f"   Text: '{text}'")
            print(f"   Prediction: {predicted_label} ({confidence:.3f})")
            print()

    except FileNotFoundError as e:
        print(f"❌ Could not load saved model: {e}")
        print("   Please run the main training pipeline first.")

if __name__ == "__main__":
    # Run the complete tutorial
    main()

    # Demonstrate model loading
    load_and_use_saved_model()